# EXAMPLE: 3 Players 2 actions with descent

In [ ]:
import numpy as np
from sage.all import *
from gtnash.game.hypergraphicalgame  import *
from gtnash.util.irda import *
from gtnash.util.polynomial_complementary_problem import *
from gtnash.solver.wilsonalgorithm import *


def present_node(current_node,y_values):
    print(f"Level: {current_node.level}")
    var_pl=[]
    var_notpl=[]
    for (n,i) in current_node.pcp.couple_to_x.keys():
        if n>current_node.level:
            var_notpl+=[current_node.pcp.couple_to_x[(n,i)]]
        else:
            var_pl+=[current_node.pcp.couple_to_x[(n,i)]]
    print(f"Variables of players at the current level: {var_pl}")
    print(f"Variables of players above the current level:{var_notpl}")
    print("Polynomials:")
    for pol in current_node.pcp.couple_to_poly.keys(): 
        print(" "+str(pol)+": "+str(current_node.pcp.couple_to_poly[pol].subs(y_values)))

    print(f"Set Z: {current_node.set_z}")
    print(f"Set W: {current_node.set_w}")
    print(f"Coordinate of the node: {current_node.coordinates}")

    print("Equations:")
    for e in range(len(current_node.sys_S.equations)):
        if not current_node.sys_S.equations[e].subs(y_values)==0:
            print(f" {current_node.sys_S.equations[e].subs(y_values)} = 0")
    print("Inequations:")
    for (n,i) in current_node.pcp.couple_to_x.keys():
        if not (n,i) in current_node.set_z:
            print(f" {current_node.pcp.couple_to_x[n,i]} >= 0")
    for e in range(len(current_node.sys_S.inequations)):
        print(f" {current_node.sys_S.inequations[e].subs(y_values)} >=0 ")


## Definition of the game:

Before creating the PCP of a game, we first compute its disutility:

In [ ]:
players_actions = [[0, 1], [0, 1], [0, 1]]
utilities=[[19, 58, 64, 66, 87, 52, 59, 79] ,
[73, 23, 80, 0, 100, 89, 43, 69] ,
[77, 60, 29, 35, 52, 98, 19, 70]  ]
current_nfg=NFG(players_actions,utilities)
print("Utilities of the game")
print(current_nfg)

label = []
for i in range (len(current_nfg.players_actions)):
    label.append('Ax'.replace("x", str(i))) #'A _i'
for i in range (len(current_nfg.players_actions),2*(len(current_nfg.players_actions))):
    label.append('ax'.replace("x", str(i-len(current_nfg.players_actions)))) #'U _i'

tableau_final = np.concatenate((np.mat(label), np.concatenate( (current_nfg.joint_actions,current_nfg.disutilities.astype(int).astype(str)),axis=1)),axis=0)
print("Disutilities of the game:")
print(tableau_final)

## PCP: Polynomials creation

Using the disutilities, we compute the polynomials associated to each couple (player, action) to create the PCP associated to this game.

Here the variables yn_i will all be equals to 1.

In [ ]:
pcpglobal = PolynomialComplementaryProblem(current_nfg)
print(pcpglobal)
#MAnual print after removing the yn_g variables?

## Choosing an arbitrary joint strategy
Before using the path following algorthim an joint strategy must be chosen between among all the possible joint strategy. Meaning that for this example we have the choice between:
* (0,0,0)
* (0,0,1)
* (0,1,0)
* (0,1,1)
* (1,0,0)
* (1,0,1)
* (1,1,0)
* (1,1,1)

Here we will choose omega0 = (0, 1, 0)

## Computing the first node:


In [ ]:

pcpglobal.omega0={0:0,1:1,2:0}
omega0=pcpglobal.omega0
xpairs_val = {}
for (n,i) in pcpglobal.couple_to_x.keys():
    if omega0[n]==i:
        xpairs_val[(n,i)] = 1
    else:
        xpairs_val[(n,i)] = 0

y_values={pcpglobal.ring("y0_0"):1, pcpglobal.ring("y1_0"):1, pcpglobal.ring("y2_0"):1}
print("The static value given to each variable xn_i for a couple (n,i) when current level< n are the following:")
print("xpairs_val:\n",xpairs_val)


# check first node computation:
node0 = first_node(pcpglobal,xpairs_val)[0]
#print("The corresponding first node is:\n",node0)
print("\nThe corresponding first node is:\n")
print(f"Level: {node0.level}")

var_pl=[]
var_notpl=[]
for (n,i) in node0.pcp.couple_to_x.keys():
    if n>node0.level:
        var_notpl+=[node0.pcp.couple_to_x[(n,i)]]
    else:
        var_pl+=[node0.pcp.couple_to_x[(n,i)]]
print(f"Variables of players at the current level: {var_pl}")
print(f"Variables of players above the current level:{var_notpl}")
print("Polynomials:")
for pol in node0.pcp.couple_to_poly.keys(): 
    print(" "+str(pol)+": "+str(node0.pcp.couple_to_poly[pol].subs(y_values)))
    
print(f"Set Z: {node0.set_z}")
print(f"Set W: {node0.set_w}")
print(f"Coordinate of the node: {node0.coordinates}")


## Complementary node at the level 0

For this node we have:
* Z=[(0,1)]
* W=[(0,0)]


Meaning that the node is compelmentary.

Given that the node is complementary at the level 0, we ascend the to level 1.


To reach the initial node at the level 1 for the node we found, we follow the arc (1,1) almost complementary where 
* Z=[(0,1),(1,0)]
* W=[(0,0)]

Because the initial joint strategy was (0,1,0) we initial must have the couple (1,0) in Z.

Considering that we are doing a lifting procedure we have two possible node we can reach, one for each possible couple (1,i) we can add to the set W:
* Z=[(0,1),(1,0)]
* W=[(0,0),(1,0)]

Or

* Z=[(0,1),(1,0)]
* W=[(0,0),(1,1)]

The node must be feasible, meaning that every value of the coordinate (variables xn_i) as well as the value of the polynomials An_i are >=0  


In [ ]:
node1=node0.lift(pcpglobal,xpairs_val)[0]

print("\nThe feasible initial node is:\n")
present_node(node1,y_values)

## Arc traversal at the level 1

The initial node is:
* Z=[(0,1),(1,0)]
* W=[(0,0),(1,1)]

The node found is complementary meaning that we can do a lift procedure to the level 2.

## Lifing procedure from the level 1 to 2

Considering the joint strategy chosen initialy we will follow the arc:
* Z=[(0,1),(1,0),(2,1)]
* W=[(0,0),(1,1)]

Wich means that we will consider the following nodes:
* Z=[(0,1),(1,0),(2,1)]
* W=[(0,0),(1,1),(2,0)]

And

* Z=[(0,1),(1,0),(2,1)]
* W=[(0,0),(1,1),(2,1)]



In [ ]:
node2=node1.lift(pcpglobal,xpairs_val)[0]

print("\nThe initial almost complementary node is:\n")
present_node(node2,y_values)

## Arc traversal at the level 2

The initial node is:
* Z=[(0,1),(1,0),(2,1)]
* W=[(0,0),(1,1),(2,1)]

The node found is almost complementary meaning that we must follow the arc that "leaves" from the node.

Given that the couple that "entered" a set was (2,1) in W, to try an reach a complementary node throught the (2,0) almost complementary path we remove (2,1) from the set Z meaning that we will follow the arc:

* Z=[(0,1),(1,0)]
* W=[(0,0),(1,1),(2,1)]


Which in turn means that the couple that can enter Z are :
* (2,0)

The couple (0,0) and (1,1) and  can't enter Z because it would correspond to a strategy where the player associated to 0 or 1 play no strategy.

And enter W are:
* (0,1)
* (1,0)
* (2,0)

Which means there is 4 possible node we can reach:

* Z=[(0,1),(1,0),(2,0)]
* W=[(0,0),(1,1),(2,1)]

,

* Z=[(0,1),(1,0)]
* W=[(0,0),(1,1),(2,1),(0,1)]

,

* Z=[(0,1),(1,0)]
* W=[(0,0),(1,1),(2,1),(1,0)]

And

* Z=[(0,1),(1,0)]
* W=[(0,0),(1,1),(2,1),(2,0)]


In [ ]:
node3=node2.traverse(node1, xpairs_val)[0]


print("\nThe almost complementary node is:\n")
present_node(node3,y_values)


## Arc traversal at the level 2

The almost complementary node is:
* Z=[(0,1),(1,0)]
* W=[(0,0),(1,1),(2,1),(1,0)]

The node found is almost complementary meaning that we must follow the arc that "leaves" from the node.

Given that the couple that "entered" a set was (1,0) in W, to try an reach a complementary node throught the (2,0) almost complementary path we remove (1,0) from the set Z meaning that we will follow the arc:

* Z=[(0,1)]
* W=[(0,0),(1,1),(2,1),(1,0)]


Which in turn means that the couple that can enter Z are :
* (1,1)
* (2,0)
* (2,1)

The couple (0,0) and  can't enter Z because it would correspond to a strategy where the player associated to 0 play no strategy.

And enter W are:
* (0,1)
* (2,0)

Which means there is 5 possible node we can reach:

* Z=[(0,1),(1,1)]
* W=[(0,0),(1,1),(2,1),(1,0)]

,

* Z=[(0,1),(2,0)]
* W=[(0,0),(1,1),(2,1),(1,0)]

,

* Z=[(0,1),(2,1)]
* W=[(0,0),(1,1),(2,1),(1,0)]

,

* Z=[(0,1)]
* W=[(0,0),(1,1),(2,1),(1,0),(0,1)]

And

* Z=[(0,1)]
* W=[(0,0),(1,1),(2,1),(1,0),(2,0)]


In [ ]:
node4=node3.traverse(node2, xpairs_val)[0]


print("\nThe almost complementary node is:\n")
present_node(node4,y_values)

## Arc traversal at the level 2

The almost complementary node is:
* Z=[(0,1)]
* W=[(0,0),(1,1),(2,1),(1,0),(0,1)]

The node found is almost complementary meaning that we must follow the arc that "leaves" from the node.

Given that the couple that "entered" a set was (0,1) in W, to try an reach a complementary node throught the (2,0) almost complementary path we remove (0,1) from the set Z meaning that we will follow the arc:

* Z=[]
* W=[(0,0),(1,1),(2,1),(1,0),(0,1)]


Which in turn means that the couple that can enter Z are :
* (0,0)
* (1,0)
* (1,1)
* (2,0)
* (2,1)


And enter W are:
* (2,0)

Which means there is 6 possible node we can reach:

* Z=[(0,0)]
* W=[(0,0),(1,1),(2,1),(1,0),(0,1)]

,

* Z=[(1,0)]
* W=[(0,0),(1,1),(2,1),(1,0),(0,1)]

,

* Z=[(1,1)]
* W=[(0,0),(1,1),(2,1),(1,0),(0,1)]

,

* Z=[(2,0)]
* W=[(0,0),(1,1),(2,1),(1,0),(0,1)]

,

* Z=[(2,1)]
* W=[(0,0),(1,1),(2,1),(1,0),(0,1)]

And

* Z=[]
* W=[(0,0),(1,1),(2,1),(1,0),(0,1),(2,0))]


In [ ]:
node5=node4.traverse(node3, xpairs_val)[0]


print("\nThe almost complementary node is:\n")
present_node(node5,y_values)


The node found is:

* Z=[(2,1)]
* W=[(0,0),(1,1),(2,1),(1,0),(0,1)]

Which is an almost complementary initial node, meaning that we must do a descent procedure

## Descent from the level 2 to level 1


We reach the complementary node of the level 1 corresponding to the initial node of the level 2 by removing all the couple including the player 2 meaning that we have the node.

* Z=[]
* W=[(0,0),(1,1),(1,0),(0,1)]

We have $Z\cap W=\emptyset$ so there is no supplementary operation to define the complementary node



In [ ]:
node6=node5.descend(node5.pcp, xpairs_val)[0]


print("\nThe complementary node is:\n")
present_node(node6,y_values)

## Arc traversal at the level 1 from a complementary node after descent

The complementary node is:
* Z=[]
* W=[(0,0),(1,1),(1,0),(0,1)]

The node found is complementary after a descent procedure meaning that we must follow the arc that "leaves" from the node towards a (1,1) almost complementary path.

The arc that follows a (1,1) almost complementary path is the one were the couple (1,1) is absent from both Z and W which is the following arc:


* Z=[]
* W=[(0,0),(1,0),(0,1)]


The couple that can enter Z are:
* (0,0)
* (0,1)
* (1,0)
* (1,1)

Which means there is 4 possible node we can reach:

* Z=[(0,0)]
* W=[(0,0),(1,0),(0,1)]

,

* Z=[(0,1)]
* W=[(0,0),(1,0),(0,1)]

,

* Z=[(1,0)]
* W=[(0,0),(1,0),(0,1)]

and 

* Z=[(1,1)]
* W=[(0,0),(1,0),(0,1)]



In [ ]:
node7=node6.traverse(node5, xpairs_val)[0]


print("\nThe almost complementary node is:\n")
present_node(node7,y_values)


## Arc traversal at the level 1

The almost complementary node is:
* Z=[(0,0)]
* W=[(0,0),(1,0),(0,1)]

The node found is almost complementary meaning that we must follow the arc that "leaves" from the node.

Given that the couple that "entered" a set was (0,0) in Z, to try an reach a complementary node throught the (1,1) almost complementary path we remove (0,0) from the set W meaning that we will follow the arc:

* Z=[(0,0)]
* W=[(1,0),(0,1)]


Which in turn means that the couple that can enter Z are :
* (1,0)
* (1,1)

The couple (0,1) and  can't enter Z because it would correspond to a strategy where the player associated to 0 play no strategy.


And enter W are:
* (1,1)

Which means there is 3 possible node we can reach:

* Z=[(0,0),(1,0)]
* W=[(1,0),(0,1)]

,

* Z=[(0,0),(1,1)]
* W=[(1,0),(0,1)]

And

* Z=[(0,0)]
* W=[(1,0),(0,1),(1,1)]

In [ ]:
node8=node7.traverse(node6, xpairs_val)[0]


print("\nThe almost complementary node is:\n")

present_node(node8,y_values)


* Z=[(0,0),(1,1)]
* W=[(1,0),(0,1)]

The node found is complementary meaning that we can do a lift procedure to the level 2.

## Lifing procedure from the level 1 to 2

Considering the joint strategy chosen initialy we will follow the arc:
* Z=[(0,0),(1,1),(2,1)]
* W=[(1,0),(0,1)]

Wich means that we will consider the following nodes:
* Z=[(0,0),(1,1),(2,1)]
* W=[(1,0),(0,1),(2,0)]

And

* Z=[(0,0),(1,1),(2,1)]
* W=[(1,0),(0,1),(2,1)]

In [ ]:
node9=node8.lift(pcpglobal, xpairs_val)[0]


print("\nThe initial node is:\n")

present_node(node9,y_values)

## Arc traversal at the level 2

The almost complementary node is:
* Z=[(0, 0), (1, 1), (2, 1)]
* W=[(1, 0), (0, 1), (2, 1)]

The node found is almost complementary meaning that we must follow the arc that "leaves" from the node.

Given that the couple that "entered" a set was (2,1) in W, to try an reach a complementary node throught the (2,0) almost complementary path we remove (2,1) from the set Z meaning that we will follow the arc:

* Z=[(0, 0), (1, 1)]
* W=[(1, 0), (0, 1), (2, 1)]


Which in turn means that the couple that can enter Z are :
* (2,0)

The couple (0,1) and (1,0)  can't enter Z because it would correspond to a strategy where the player associated to 0 or 1 plays no strategy.



And enter W are:
* (0,0)
* (1,1)
* (2,0)



Which means there is 4 possible node we can reach:

* Z=[(0, 0), (1, 1), (2, 0)]
* W=[(1, 0), (0, 1), (2, 1)]

,

* Z=[(0, 0), (1, 1)]
* W=[(1, 0), (0, 1), (2, 1), (0, 0)]

,

* Z=[(0, 0), (1, 1)]
* W=[(1, 0), (0, 1), (2, 1), (1, 1)]

And

* Z=[(0, 0), (1, 1)]
* W=[(1, 0), (0, 1), (2, 1), (2, 0)]


In [ ]:
node10=node9.traverse(node8, xpairs_val)[0]


print("\nThe almost complementary node is:\n")

present_node(node10,y_values)

## Arc traversal at the level 2

The almost complementary node is:
* Z=[(0, 0), (1, 1)]
* W=[(1, 0), (0, 1), (2, 1), (0, 0)]

The node found is almost complementary meaning that we must follow the arc that "leaves" from the node.

Given that the couple that "entered" a set was (0,0) in W, to try an reach a complementary node throught the (2,0) almost complementary path we remove (0,0) from the set Z meaning that we will follow the arc:

* Z=[(1, 1)]
* W=[(1, 0), (0, 1), (2, 1), (0, 0)]


Which in turn means that the couple that can enter Z are :
* (0, 1)
* (2, 0)
* (2, 1)

The couple (1,0)  can't enter Z because it would correspond to a strategy where the player associated to 1 plays no strategy.



And enter W are:
* (1,1)
* (2,0)



Which means there is 5 possible node we can reach:

* Z=[(1, 1), (0, 1)]
* W=[(1, 0), (0, 1), (2, 1), (0, 0)]

,

* Z=[(1, 1), (2, 0)]
* W=[(1, 0), (0, 1), (2, 1), (0, 0)]

,

* Z=[(1, 1), (2, 1)]
* W=[(1, 0), (0, 1), (2, 1), (0, 0)]

,

* Z=[(1, 1)]
* W=[(1, 0), (0, 1), (2, 1), (0, 0), (1, 1)]

And

* Z=[(1, 1)]
* W=[(1, 0), (0, 1), (2, 1), (0, 0), (2, 0)]


In [ ]:
node11=node10.traverse(node9, xpairs_val)[0]


print("\nThe almost complementary node is:\n")

present_node(node11,y_values)

## Complementary node found

* Z=[(1, 1)]
* W=[(1, 0), (0, 1), (2, 1), (0, 0), (2, 0)]

The node found is complementary and we are at the last level, meaning that this node is a solution to the PCP which correspond to mixed nash equilibrium of the corresponding game.

The coordinate of the nodes are:

In [ ]:
for (n,i) in pcpglobal.couple_to_x.keys():
    print(f"{pcpglobal.couple_to_x[(n,i)]} = {node11.coordinates[pcpglobal.couple_to_x[(n,i)]]} = {node11.coordinates[pcpglobal.couple_to_x[(n,i)]].radical_expression()}")

Which when normalized give us the following mixed strategy:

In [ ]:
mixed_strat=node11.normalized_strategy()
for n in mixed_strat.keys():
    print(f"Player {n}: {mixed_strat[n]}")